In [3]:
import pandas as pd
import zipfile
import plotly.graph_objects as go

In [4]:
# Create a ZipFile object
with zipfile.ZipFile('dataframes.zip', 'r') as zipf:
    # Get a list of all CSV file names in the zip file
    csv_files = [name for name in zipf.namelist() if name.endswith('.csv')]
    
    # Read each CSV file into a DataFrame and store them in a list
    dataframes = [pd.read_csv(zipf.open(csv_file)) for csv_file in csv_files]

In [5]:
def create_frame(day_index, df):
    df_melted = df.melt(id_vars=['Unnamed: 0'], var_name='Longitude', value_name='Temperature')
    df_melted.columns = ['Latitude', 'Longitude', 'Temperature']
    df_melted.dropna(inplace=True)
    df_melted['Latitude'] = df_melted['Latitude'].astype(float)
    df_melted['Longitude'] = df_melted['Longitude'].astype(float)
    df_melted['Temperature'] = df_melted['Temperature'].astype(float)
    
    scatter = go.Scattergeo(
        lat=df_melted['Latitude'],
        lon=df_melted['Longitude'],
        text=df_melted['Temperature'],
        mode='markers',
        marker=dict(
            size=5,
            color=df_melted['Temperature'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Temperature")
        )
    )
    
    frame = go.Frame(data=[scatter], name=str(day_index))
    return frame

In [ ]:
# Initialize figure with the layout
fig = go.Figure()

# Add frames for each day
frames = [create_frame(i, df) for i, df in enumerate(dataframes)]
fig.frames = frames

# Ensure there's data for the first frame to display initially
if frames:
    fig.add_trace(frames[0].data[0])

# Add a slider to control which frame is displayed
sliders = [{
    'steps': [{'method': 'animate', 'args': [[frame.name], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate'}], 'label': str(i)} for i, frame in enumerate(fig.frames)],
    'transition': {'duration': 300},
    'x': 0.1,
    'len': 0.9,
    'xanchor': 'left',
    'y': 0,
    'yanchor': 'top'
}]

fig.update_layout(
    sliders=sliders,
    geo=dict(
        scope='usa',
        projection=go.layout.geo.Projection(type='albers usa')
    ),
    updatemenus=[{
        'type': 'buttons',
        'showactive': False,
        'y': 0.8,
        'x': 0.0,
        'xanchor': 'right',
        'yanchor': 'top',
        'pad': {'t': 60, 'r': 10},
        'buttons': [
            {
                'label': '▶',
                'method': 'animate',
                'args': [None, {
                    'frame': {'duration': 500, 'redraw': True},
                    'fromcurrent': True,
                    'transition': {'duration': 300, 'easing': 'quadratic-in-out'}
                }]
            },
            {
                'label': '⏸',
                'method': 'animate',
                'args': [[None], {
                    'mode': 'immediate',
                    'frame': {'duration': 0, 'redraw': False},
                }]
            },
            {
                'label': '⏮',
                'method': 'animate',
                'args': [[frames[0].name], {
                    'frame': {'duration': 0, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 0}
                }]
            }
        ]
    }]
)

# Show figure
fig.show()